# 【課題】位相推定によるスペクトル分解



In [ ]:
# まず必要なモジュールをインポートする
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram
# ワークブック独自のモジュール
from utils.hamiltonian import make_hamiltonian
from utils.show_state import show_state, show_statevector

print('notebook ready')

In [ ]:
# Number of spins
n_s = 2
# Coupling parameter
J = 1.
# External field / J
g = 0.

# Construct the Hamiltonian matrix
paulis = list()
coeffs = list()
for j in range(n_s):
    paulis.append(list('x' if k in (j, (j + 1) % n_s) else 'i' for k in range(n_s)))
    coeffs.append(-J)
    paulis.append(list('y' if k in (j, (j + 1) % n_s) else 'i' for k in range(n_s)))
    coeffs.append(-J)
    paulis.append(list('z' if k in (j, (j + 1) % n_s) else 'i' for k in range(n_s)))
    coeffs.append(-J)
    if g != 0.:
        paulis.append(list('z' if k == j else 'i' for k in range(n_s)))
        coeffs.append(-J * g)

hamiltonian = make_hamiltonian(paulis, coeffs)

# Diagonalize and obtain the eigenvalues and vectors
eigvals, eigvectors = np.linalg.eigh(hamiltonian)

# Print the eigenvectors
for i in range(eigvals.shape[0]):
    show_statevector(eigvectors[:, i], binary=True, state_label=r'\phi_{} (E={}J)'.format(i, eigvals[i]))


## 問題1: スペクトル推定を実装し、厳密解と照らし合わせる

In [ ]:
def make_trotter_step_heisenberg(n, g, hbar_omega=None):
    """Return a function that implements a single Trotter step for the Heisenberg model.
    
    Args:
        n (int): Number of spins in the model.
        g (float): External field strength relative to the coupling constant J.
        hbar_omega (None or float): If not None, provides a custom Hamiltonian normalization in units of J.
        
    Returns:
        function(dphi: float)->QuantumCircuit: A function that returns a circuit corresponding to a single Trotter step by dphi.
    """

    if hbar_omega is None:
        # H = -J sum[XX + YY + ZZ + gZ]
        # |sum[]| < (3 + |g|) * n
        # -> hbar * omega = 2 * (3 + |g|) * n * J
        #    Theta = -1 / 2 / [(3 + |g|) * n] * sum[]
        hbar_omega = 2. * (3. + abs(g)) * n
        
    def trotter_step_heisenberg(dphi):
        """
        Evolve the state by exp[i * dphi * Theta].
        Theta = H / (hbar * omega) is the normalized Hamiltonian.
    
        Args:
            dphi (float): Trotter step size.
            
        Returns:
            QuantumCircuit: A circuit corresponding to a single Trotter step.
        """
        
        circuit = QuantumCircuit(n)
        
        ##################
        ### EDIT BELOW ###
        ##################

        #circuit.?

        ##################
        ### EDIT ABOVE ###
        ##################
        
        return circuit
    
    return trotter_step_heisenberg

In [ ]:
def propagator(trotter_step, power, num_steps=6):
    """Call trotter_step(2 * pi / num_steps) for power * num_steps times and convert the circuit to a gate.
    
    Args:
        trotter_step (callable): A function implementing a single Trotter step.
        power (int): Number of repetitions of the 2pi evolution.
        num_steps (int): Number of Trotter steps per 2pi evolution.
        
    Returns:
        Gate: Propagator circuit converted to a gate.
    """
    
    circuit = QuantumCircuit(name='propagator^{}'.format(power))
    circuit += trotter_step(2. * np.pi / num_steps).repeat(power * num_steps)
    return circuit.to_gate()

def spectrum_estimation(circuit, trotter_step):
    """Perform a spectrum estimation given a circuit containing state and readout registers and a callable implementing
    a single Trotter step.
    
    Args:
        circuit (QuantumCircuit): Circuit with two registers 'state' and 'readout'.
        trotter_step (callable): A function returning a QuantumCircuit corresponding to a Trotter step.
    """
    
    state_register = next(reg for reg in circuit.qregs if reg.name == 'state')
    readout_register = next(reg for reg in circuit.qregs if reg.name == 'readout')
    
    # Set the R register to an equal superposition
    circuit.h(readout_register)

    # Apply controlled-U operations to the circuit
    for qubit in readout_register:
        # Create a gate from repeated execution of the Trotter step, and convert it to a controlled gate
        controlled_u = propagator(trotter_step, 2 ** qubit.index).control(1)
        # Append the controlled gate specifying the control and target qubits
        circuit.append(controlled_u, qargs=([qubit] + state_register[:]))

    # Inverse QFT
    for j in range(readout_register.size // 2):
        circuit.swap(readout_register[j], readout_register[-1 - j])

    dphi = 2. * np.pi / (2 ** readout_register.size)

    for jtarg in range(readout_register.size):
        for jctrl in range(jtarg):
            power = jctrl - jtarg - 1 + readout_register.size
            circuit.cp(-dphi * (2 ** power), readout_register[jctrl], readout_register[jtarg])

        circuit.h(readout_register[jtarg])

In [ ]:
n_state = 2
n_readout = 4
g = 0.
hbar_omega = 16. # 16J but we are setting J=1

state_register = QuantumRegister(n_state, 'state')
readout_register = QuantumRegister(n_readout, 'readout')
circuit = QuantumCircuit(state_register, readout_register)

# Set the initial state of the state vector to (1/2)|00> - (1/sqrt(2))|01> + (1/2)|11>
##################
### EDIT BELOW ###
##################

#circuit.?

##################
### EDIT ABOVE ###
##################

trotter_step = make_trotter_step_heisenberg(n_state, g, hbar_omega=hbar_omega)

spectrum_estimation(circuit, trotter_step)
    
circuit.measure_all()

# Run the circuit in qasm_simulator and plot the histogram
qasm_simulator = Aer.get_backend('qasm_simulator')
job = execute(circuit, backend=qasm_simulator, shots=10000)
result = job.result()
counts = result.get_counts(circuit)
plot_histogram(counts)

**提出するもの**

- 完成した`make_trotter_step_heisenberg`関数
- 完成した状態レジスタの初期化回路
- スペクトル推定の結果のヒストグラムと、その解釈

## 問題2: 非自明な系の振る舞いを調べる

In [ ]:
def get_spectrum_for_comp_basis(n_state, n_readout, l, g, hbar_omega=None, use_qasm=False):
    """Compute and return the distribution P_l(k, h) as an ndarray.
    
    Args:
        n_state (int): Size of the state register.
        n_readout (int): Size of the readout register.
        l (int): Index of the initial-state computational basis in the state register.
        g (float): Parameter g of the Heisenberg model.
        hbar_omega (None or float): Hamiltonian normalization.
        use_qasm (bool): Use the qasm_simulator if True.
    """
    
    # Define the circuit
    state_register = QuantumRegister(n_state, 'state')
    readout_register = QuantumRegister(n_readout, 'readout')
    circuit = QuantumCircuit(state_register, readout_register)

    # Initialize the state register
    for iq in range(n_state):
        if ((l >> iq) & 1) == 1:
            circuit.x(state_register[iq])

    # Run the spectrum estimation
    trotter_step = make_trotter_step_heisenberg(n_state, g, hbar_omega=hbar_omega)
    spectrum_estimation(circuit, trotter_step)

    # Extract the probability distribution as an array of shape (2 ** n_readout, 2 ** n_state)
    if use_qasm:
        circuit.measure_all()

        # Run the circuit in qasm_simulator and plot the histogram
        qasm_simulator = Aer.get_backend('qasm_simulator')
        job = execute(circuit, backend=qasm_simulator, shots=10000)
        result = job.result()
        counts = result.get_counts(circuit)

        probs = np.zeros((2 ** n_readout, 2 ** n_state), dtype=float)

        for bitstring, count in counts.items():
            readout = int(bitstring[:n_readout], 2)
            state = int(bitstring[n_readout:], 2)

            probs[readout, state] = count

        probs /= np.sum(probs)
    
    else:
        sv_simulator = Aer.get_backend('statevector_simulator')
        job = execute(circuit, backend=sv_simulator)
        result = job.result()
        statevector = result.data()['statevector']
        
        # Convert the state vector into a probability distribution by taking the norm-squared
        probs = np.square(np.abs(statevector)).reshape((2 ** n_readout, 2 ** n_state))
        # Clean up the numerical artifacts
        probs = np.where(probs > 1.e-6, probs, np.zeros_like(probs))
    
    # probs[k, h] = P_l(k, h)
    return probs

In [ ]:
n_state = 4
n_readout = 5
hbar_omega = 24.

g_values = np.linspace(0., 0.5, 6, endpoint=True)

spectra = np.empty((g_values.shape[0], 2 ** n_readout), dtype=float)

def get_full_spectrum(g):
    """Compute and return the distribution P(k) for a value of g.
    """

    spectrum = np.zeros(2 ** n_readout, dtype=float)
    
    for l in range(2 ** n_state):
        probs = get_spectrum_for_comp_basis(n_state, n_readout, l, g, hbar_omega=hbar_omega)
        print('Computed spectrum for g = {:.1f} l = {:d}'.format(g, l))

        ##################
        ### EDIT BELOW ###
        ##################

        ##################
        ### EDIT ABOVE ###
        ##################
        
    return spectrum

# roll(spectrum, 2^{n_R-1}) => range of k is [-2^{n_R}/2, 2^{n_R}/2 - 1]
spectra[0] = np.roll(get_full_spectrum(0.), 2 ** (n_readout - 1))

In [ ]:
plt.plot(np.linspace(-0.5 * hbar_omega, 0.5 * hbar_omega, 2 ** n_readout), spectra[0], 'o')
plt.xlabel('E/J')
plt.ylabel('P(E)')

In [ ]:
for i in range(1, g_values.shape[0]):
    spectra[i] = np.roll(get_full_spectrum(g_values[i]), 2 ** (n_readout - 1))

In [ ]:
energy_eigenvalues = np.empty((g_values.shape[0], 2 ** n_state))

# Extract the energy eigenvalues from spectra and fill the array
##################
### EDIT BELOW ###
##################

#energy_eigenvalues[ig, m] = E_ig_m

##################
### EDIT ABOVE ###
##################

plt.plot(g_values, energy_eigenvalues)

**提出するもの**

- $P_l(k, h)$から$P(k)$を導出する方法の説明と、`get_full_spectrum`関数への実装
- $P(k)$からエネルギー固有値を抽出する方法を考案し、実装したコード（エネルギー固有値を求める全く別の方法を考えつけば、それでもよい）
- 16個のエネルギー固有値と$g$の関係のプロット